In [2]:
import cv2
from PIL import Image, ImageOps
import keras
import numpy as np
#import imgaug as ia
#import imgaug.augmenters as iaa
import requests
from io import BytesIO
from keras.models import load_model, Model
from keras.layers import Flatten, Dense, GlobalAveragePooling2D
import keras.models
import keras.optimizers
import tensorflow as tf
from keras.models import load_model
from PIL import Image
import glob
from keras.layers import Flatten, Dense
import tensorflow.keras.metrics
import tensorflow.keras.backend as K
from sklearn.metrics import classification_report
import datetime
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [MPG]')
    plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='mean_absolute_error')
    plt.plot(hist['epoch'], hist['mean_squared_error'],
           label = 'mean_squared_error')
    plt.ylim([0,0.4])
    plt.legend()
    plt.show()
    
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def create_model():
    model = keras.applications.densenet.DenseNet121(include_top=False, weights='imagenet', pooling='avg')
    X = model.output
    output = Dense(11, activation = 'softmax')(X)
    model = Model(inputs = model.inputs, outputs = output)   
    opt = keras.optimizers.Adam(lr=0.001, decay = 1e-6, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1_m])
    return model




In [3]:
X_train = np.load('image_train.npy')/255.0
y_train = np.load('target_train.npy')
#X_test = np.load('image_test.npy')/255.0
#y_test = np.load('target_test.npy')


In [4]:
y_train = keras.utils.to_categorical(y_train - 1, dtype='float32')
#y_test = keras.utils.to_categorical(y_test -1 , dtype='float32')

In [8]:
model = create_model()

29089792/29084464 [==============================] - 22s 1us/step


In [ ]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir='log')
es = keras.callbacks.EarlyStopping(monitor=f1_m, patience=15, verbose=0, mode='min',  restore_best_weights=True)
history = model.fit(X_train, y_train, epochs = 200, batch_size = 32, verbose = 1,shuffle = True, callbacks = [tensorboard_callback, es])

In [5]:
model_name = 'mobile_net_7783.h5'
model = load_model(model_name, custom_objects= {'f1_m' : f1_m, 'precision' : precision_m, 'recall_m' : recall_m})

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [7]:
model.save('mobile_net_7783.h5')